In [51]:
%%sql
postgresql:///games

-- Select all information for the top ten best-selling games
-- Order the results from best-selling game down to tenth best-selling

SELECT *
    FROM game_sales
    ORDER BY games_sold DESC
    LIMIT 10;

10 rows affected.


game,platform,publisher,developer,games_sold,year
Wii Sports for Wii,Wii,Nintendo,Nintendo EAD,82.90,2006
Super Mario Bros. for NES,NES,Nintendo,Nintendo EAD,40.24,1985
Counter-Strike: Global Offensive for PC,PC,Valve,Valve Corporation,40.00,2012
Mario Kart Wii for Wii,Wii,Nintendo,Nintendo EAD,37.32,2008
PLAYERUNKNOWN'S BATTLEGROUNDS for PC,PC,PUBG Corporation,PUBG Corporation,36.60,2017
Minecraft for PC,PC,Mojang,Mojang AB,33.15,2010
Wii Sports Resort for Wii,Wii,Nintendo,Nintendo EAD,33.13,2009
Pokemon Red / Green / Blue Version for GB,GB,Nintendo,Game Freak,31.38,1998
New Super Mario Bros. for DS,DS,Nintendo,Nintendo EAD,30.80,2006
New Super Mario Bros. Wii for Wii,Wii,Nintendo,Nintendo EAD,30.30,2009


In [53]:
%%sql 

-- Join games_sales and reviews
-- Select a count of the number of games where both critic_score and user_score are null

SELECT COUNT(g.game)
FROM game_sales g
LEFT JOIN reviews r
ON g.game = r.game
WHERE critic_score IS NULL AND user_score IS NULL;

 * postgresql:///games
1 rows affected.


count
31


In [55]:
%%sql

-- Select release year and average critic score for each year, rounded and aliased
-- Join the game_sales and reviews tables
-- Group by release year
-- Order the data from highest to lowest avg_critic_score and limit to 10 results

SELECT g.year, ROUND(AVG(r.critic_score),2) AS avg_critic_score
FROM game_sales g
INNER JOIN reviews r
ON g.game = r.game
GROUP BY g.year
ORDER BY avg_critic_score DESC
LIMIT 10;

 * postgresql:///games
10 rows affected.


year,avg_critic_score
1990,9.80
1992,9.67
1998,9.32
2020,9.20
1993,9.10
1995,9.07
2004,9.03
1982,9.00
2002,8.99
1999,8.93


In [57]:
%%sql 

-- Paste query from the previous task; update it to add a count of games released in each year called num_games
-- Update the query so that it only returns years that have more than four reviewed games

SELECT g.year, COUNT(g.game) AS num_games, ROUND(AVG(r.critic_score),2) AS avg_critic_score
FROM game_sales g
INNER JOIN reviews r
ON g.game = r.game
GROUP BY g.year
HAVING COUNT(g.game) > 4
ORDER BY avg_critic_score DESC
LIMIT 10;

 * postgresql:///games
10 rows affected.


year,num_games,avg_critic_score
1998,10,9.32
2004,11,9.03
2002,9,8.99
1999,11,8.93
2001,13,8.82
2011,26,8.76
2016,13,8.67
2013,18,8.66
2008,20,8.63
2012,12,8.62


In [59]:
%%sql 

-- Select the year and avg_critic_score for those years that dropped off the list of critic favorites 
-- Order the results from highest to lowest avg_critic_score

SELECT year, avg_critic_score
FROM top_critic_years
EXCEPT
SELECT year, avg_critic_score
FROM top_critic_years_more_than_four_games
ORDER BY avg_critic_score DESC;

 * postgresql:///games
6 rows affected.


year,avg_critic_score
1990,9.80
1992,9.67
2020,9.20
1993,9.10
1995,9.07
1982,9.00


In [61]:
%%sql 

-- Select year, an average of user_score, and a count of games released in a given year, aliased and rounded
-- Include only years with more than four reviewed games; group data by year
-- Order data by avg_user_score, and limit to ten results

SELECT g.year, COUNT(g.game) AS num_games, ROUND(AVG(r.user_score),2) AS avg_user_score
FROM game_sales g
INNER JOIN reviews r
ON g.game = r.game
GROUP BY g.year
HAVING COUNT(g.game) > 4
ORDER BY avg_user_score DESC
LIMIT 10;

 * postgresql:///games
10 rows affected.


year,num_games,avg_user_score
1997,8,9.50
1998,10,9.40
2010,23,9.24
2009,20,9.18
2008,20,9.03
1996,5,9.00
2005,13,8.95
2006,16,8.95
2000,8,8.80
1999,11,8.80


In [63]:
%%sql 

-- Select the year results that appear on both tables

SELECT year
FROM top_user_years_more_than_four_games
INTERSECT
SELECT year
FROM top_critic_years_more_than_four_games

 * postgresql:///games
3 rows affected.


year
1998
2008
2002


In [65]:
%%sql 

-- Select year and sum of games_sold, aliased as total_games_sold; order results by total_games_sold descending
-- Filter game_sales based on whether each year is in the list returned in the previous task

SELECT g.year, SUM(g.games_sold) AS total_games_sold
FROM game_sales g
WHERE g.year IN (SELECT year
FROM top_user_years_more_than_four_games
INTERSECT
SELECT year
FROM top_critic_years_more_than_four_games)
GROUP BY g.year
ORDER BY total_games_sold DESC;

 * postgresql:///games
3 rows affected.


year,total_games_sold
2008,175.07
1998,101.52
2002,58.67
